In [1]:
import torch
from pathlib import Path
import itertools
import numpy as np
import time
import random
from random import randrange
    
###Test Streaming DataLoader with PyTorch####
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, filePaths, frameLength, hopSize, loadAll = False):
            super(MyIterableDataset).__init__()
            self.loadAll = loadAll
            self.filePaths = self.group_recordings(filePaths)
            self.frameLength = frameLength
            self.hopSize = hopSize
            self.filePage = len(self.filePaths)
            self.filePool = list(range(self.filePage))
            print(self.filePage)
            random.shuffle(self.filePool)

            self.currentFileIndx = 0
            self.CurrentEEG = []
            self.CurrentAudio = []
            self.samplePosistions = []

            self.currentSampleIndex = 0
            self.loadDataToBuffer(self.currentFileIndx,loadAll)
            
            self.samplePosMap = []
            self.generateSamplePostion()
            
    def group_recordings(self, files):
        #Group recordings and corresponding stimuli.
        new_files = []
        grouped = itertools.groupby(sorted(files), lambda x: "_-_".join(x.stem.split("_-_")[:3]))
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
        return new_files    
    import random
    
    def loadDataToBuffer(self,fileIndex,loadAll):
        if loadAll == True:
            for i in range(self.filePage):
                self.CurrentEEG.append(np.load(self.filePaths[self.filePool[self.currentFileIndx]][0]).astype(np.float32))
                self.CurrentAudio.append(np.load(self.filePaths[self.filePool[self.currentFileIndx]][1]).astype(np.float32))
        else:
            self.CurrentEEG = np.load(self.filePaths[self.filePool[self.currentFileIndx]][0]).astype(np.float32)
            self.CurrentAudio = np.load(self.filePaths[self.filePool[self.currentFileIndx]][1]).astype(np.float32)

    
    def generateSamplePostion(self):
        count = 0
        if self.loadAll == True:
            for i in range(self.filePage):
                totalLength,_ = self.CurrentAudio[i].shape
                startPos = [*range(self.frameLength, totalLength+1, self.hopSize)]
                self.samplePosMap.append(startPos)
                noData = (totalLength-self.frameLength)//self.hopSize + 1
                assert len(startPos)==noData
                count += noData
            return count
        else:
            for i in range(self.filePage):
                tempAudio = np.load(self.filePaths[i][1]).astype(np.float32)
                totalLength,_ = tempAudio.shape
                startPos = [*range(self.frameLength, totalLength+1, self.hopSize)]
                self.samplePosMap.append(startPos)
                noData = (totalLength-self.frameLength)//self.hopSize + 1
                assert len(startPos)==noData
                count += noData
            return count
    def sample_random_data_number_in_one_batch(self,n, total):
    #Return a randomly chosen list of n nonnegative integers summing to total.
    #n: the number of total files    total: batch size
        return [x - 1 for x in self.constrained_sum_sample_pos(n, total + n)]
    
    def constrained_sum_sample_pos(self,n, total):
    #Return a randomly chosen list of n positive integers summing to total.Each such list is equally likely to occur."""
        dividers = sorted(random.sample(range(1, total), n - 1))
        return [a - b for a, b in zip(dividers + [total], [0] + dividers)]
            
    def __iter__(self):
       
        return self
    
    def __next__(self):
        if self.loadAll == True:
            if self.currentSampleIndex < len(self.samplePosMap[self.filePool[self.currentFileIndx]]): # still in the same file
                thisEnd = self.samplePosMap[self.filePool[self.currentFileIndx]][self.currentSampleIndex]
                self.currentSampleIndex += 1
                return self.CurrentEEG[self.filePool[self.currentFileIndx]][thisEnd-self.frameLength:thisEnd,:], self.CurrentAudio[self.filePool[self.currentFileIndx]][thisEnd-self.frameLength:thisEnd,:]
            else: # move to the next file
                #print("next file")
                #### need to shuffle samples from the last file
                random.shuffle(self.samplePosMap[self.filePool[self.currentFileIndx]])
                self.currentFileIndx +=1
                self.currentSampleIndex = 0
                if self.currentFileIndx < self.filePage: # still in the same iteration
                    #self.loadDataToBuffer(self.currentFileIndx)
                    thisEnd = self.samplePosMap[self.filePool[self.currentFileIndx]][self.currentSampleIndex]
                    self.currentSampleIndex += 1
                    return self.CurrentEEG[self.filePool[self.currentFileIndx]][thisEnd-self.frameLength:thisEnd,:], self.CurrentAudio[self.filePool[self.currentFileIndx]][thisEnd-self.frameLength:thisEnd,:]
                else:
                    #print("here 2")
                    random.shuffle(self.filePool)
                    self.currentFileIndx = 0
                    #self.loadDataToBuffer(self.currentFileIndx)
                    raise StopIteration
                    print("iteration done, restart")
        else:
            if self.currentSampleIndex < len(self.samplePosMap[self.filePool[self.currentFileIndx]]): # still in the same file
                thisEnd = self.samplePosMap[self.filePool[self.currentFileIndx]][self.currentSampleIndex]
                self.currentSampleIndex += 1
                return self.CurrentEEG[thisEnd-self.frameLength:thisEnd,:], self.CurrentAudio[thisEnd-self.frameLength:thisEnd,:]
            else: # move to the next file
                #print("next file")
                #### need to shuffle samples from the last file
                random.shuffle(self.samplePosMap[self.filePool[self.currentFileIndx]])
                self.currentFileIndx +=1
                self.currentSampleIndex = 0
                if self.currentFileIndx < self.filePage: # still in the same iteration
                    self.loadDataToBuffer(self.currentFileIndx)
                    thisEnd = self.samplePosMap[self.filePool[self.currentFileIndx]][self.currentSampleIndex]
                    self.currentSampleIndex += 1
                    return self.CurrentEEG[thisEnd-self.frameLength:thisEnd,:], self.CurrentAudio[thisEnd-self.frameLength:thisEnd,:]
                else:
                    #print("here 2")
                    random.shuffle(self.filePool)
                    self.currentFileIndx = 0
                    self.loadDataToBuffer(self.currentFileIndx)
                    raise StopIteration
                    print("iteration done, restart")

In [2]:
class CustomAllLoadDataset(torch.utils.data.Dataset):
    def __init__(self, filePaths, frameLength, hopSize):
            self.filePaths = self.group_recordings(filePaths)
            self.frameLength = frameLength
            self.hopSize = hopSize
            self.filePage = len(self.filePaths)
            # load all data to memory as continuous data
            self.EEGData = []
            self.AudioData = []
            self.loadDataToBuffer()
            
            self.sampleIndxMap = []
            self.noData = self.generateSamplePostion()
            print("total number of training examples:", self.noData)
            
    def group_recordings(self, files):
        #Group recordings and corresponding stimuli.
        new_files = []
        grouped = itertools.groupby(sorted(files), lambda x: "_-_".join(x.stem.split("_-_")[:3]))
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
        return new_files         
    
    def loadDataToBuffer(self):
        for i in range(self.filePage):
            self.EEGData.append(np.load(self.filePaths[i][0]).astype(np.float32))
            self.AudioData.append(np.load(self.filePaths[i][1]).astype(np.float32))
    
    def generateSamplePostion(self):
        count = 0
        for i in range(self.filePage):
            totalLength,_ = self.AudioData[i].shape
            startPos = [*range(self.frameLength, totalLength+1, self.hopSize)]
            for pos in startPos:
                self.sampleIndxMap.append((i, pos))
            noData = (totalLength-self.frameLength)//self.hopSize + 1
            assert len(startPos)==noData
            count += noData
        return count
        
    def __len__(self):
        return self.noData
    
    def __getitem__(self, idx):      
        return self.EEGData[self.sampleIndxMap[idx][0]][self.sampleIndxMap[idx][1]-self.frameLength:self.sampleIndxMap[idx][1],:], self.AudioData[self.sampleIndxMap[idx][0]][self.sampleIndxMap[idx][1]-self.frameLength:self.sampleIndxMap[idx][1],:]

In [3]:
import os
import argparse
import json

import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter


from util import rescale, find_max_epoch, print_size
from util import training_loss, calc_diffusion_hyperparams

from distributed_util import init_distributed, apply_gradient_allreduce, reduce_tensor

from models2 import EEGWav_diff as WaveNet

def train(window_length,hop_length, num_gpus, rank, group_name, output_directory, tensorboard_directory,
          ckpt_iter, n_iters, iters_per_ckpt, iters_per_logging,
          learning_rate, batch_size_per_gpu):
    """
    Parameters:
    num_gpus, rank, group_name:     parameters for distributed training
    output_directory (str):         save model checkpoints to this path
    tensorboard_directory (str):    save tensorboard events to this path
    ckpt_iter (int or 'max'):       the pretrained checkpoint to be loaded;
                                    automitically selects the maximum iteration if 'max' is selected
    n_iters (int):                  number of iterations to train, default is 1M
    iters_per_ckpt (int):           number of iterations to save checkpoint,
                                    default is 10k, for models with residual_channel=64 this number can be larger
    iters_per_logging (int):        number of iterations to save training log, default is 100
    learning_rate (float):          learning rate
    batch_size_per_gpu (int):       batchsize per gpu, default is 2 so total batchsize is 16 with 8 gpus
    """

    # generate experiment (local) path
    local_path = "ch{}_T{}_betaT{}".format(wavenet_config["res_channels"],
                                           diffusion_config["T"],
                                           diffusion_config["beta_T"])
    # Create tensorboard logger.
    if rank == 0:
        tb = SummaryWriter(os.path.join('exp', local_path, tensorboard_directory))

    # distributed running initialization
    if num_gpus > 1:
        init_distributed(rank, num_gpus, group_name, **dist_config)

    # Get shared output_directory ready
    output_directory = os.path.join('exp', local_path, output_directory)
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        print("output directory", output_directory, flush=True)

    # map diffusion hyperparameters to gpu
    for key in diffusion_hyperparams:
        if key is not "T":
            diffusion_hyperparams[key] = diffusion_hyperparams[key].cuda()

    # predefine model
    net = WaveNet(**wavenet_config).cuda()
    print_size(net)

    # apply gradient all reduce
    if num_gpus > 1:
        net = apply_gradient_allreduce(net)

    # define optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    # load checkpoint
    if ckpt_iter == 'max':
        ckpt_iter = find_max_epoch(output_directory)
    if ckpt_iter >= 0:
        try:
            # load checkpoint file
            model_path = os.path.join(output_directory, '{}.pkl'.format(ckpt_iter))
            checkpoint = torch.load(model_path, map_location='cpu')

            # feed model dict and optimizer state
            net.load_state_dict(checkpoint['model_state_dict'])
            if 'optimizer_state_dict' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

            print('Successfully loaded model at iteration {}'.format(ckpt_iter))
        except:
            ckpt_iter = -1
            print('No valid checkpoint model found, start training from initialization.')
    else:
        ckpt_iter = -1
        print('No valid checkpoint model found, start training from initialization.')
    
    # Get the path to the config file
    experiments_folder = "C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression"
    task_folder = Path(experiments_folder)
    config_path = task_folder/ "util/config.json"

    with open(config_path) as fp:
        config = json.load(fp)

    data_folder = Path(config["dataset_folder"])/ config["split_folder"]
    stimulus_features = ["envelope"]
    features = ["eeg"] + stimulus_features

    train_files = [path for path in Path(data_folder).resolve().glob("train_-_*") if path.stem.split("_-_")[-1].split(".")[0] in features]
    train_dataset = CustomAllLoadDataset(train_files,window_length,hop_length)

    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=8,shuffle=True)
        
    # training
    n_iter = ckpt_iter + 1
    while n_iter < n_iters + 1:
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            eeg, audio = data[0].squeeze(1).cuda(), data[1].type(torch.LongTensor).cuda()
            #print(eeg.shape,audio.shape)
            # load audio and mel spectrogram
            # mel_spectrogram = mel_spectrogram.cuda()
            # audio = audio.unsqueeze(1).cuda()

            # back-propagation
            optimizer.zero_grad()
            X = (eeg.float(), audio.float())
            loss = training_loss(net, nn.MSELoss(), X, diffusion_hyperparams)
            # print(loss)
            if num_gpus > 1:
                reduced_loss = reduce_tensor(loss.data, num_gpus).item()
            else:
                reduced_loss = loss.item()
            loss.backward()
            optimizer.step()

            # output to log
            # note, only do this on the first gpu
            if n_iter % iters_per_logging == 0 and rank == 0:
                # save training loss to tensorboard
                print("iteration: {} \treduced loss: {} \tloss: {}".format(n_iter, reduced_loss, loss.item()))
                tb.add_scalar("Log-Train-Loss", torch.log(loss).item(), n_iter)
                tb.add_scalar("Log-Train-Reduced-Loss", np.log(reduced_loss), n_iter)

            # save checkpoint
            if n_iter > 0 and n_iter % iters_per_ckpt == 0 and rank == 0:
                checkpoint_name = '{}.pkl'.format(n_iter)
                torch.save({'model_state_dict': net.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict()},
                           os.path.join(output_directory, checkpoint_name))
                print('model at iteration %s is saved' % n_iter)

            n_iter += 1

    # Close TensorBoard.
    if rank == 0:
        tb.close()



<>:58: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:58: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [4]:
"""Example experiment for a linear baseline method."""
import glob
import json
import logging
import os
import tensorflow as tf
from torch.utils.data import DataLoader
from task2_regression.models.linear import simple_linear_model
from task2_regression.util.dataset_generator import RegressionDataGenerator, create_tf_dataset
from pathlib import Path

def evaluate_model(model, test_dict):
    """Evaluate a model.

    Parameters
    ----------
    model: tf.keras.Model
        Model to evaluate.
    test_dict: dict
        Mapping between a subject and a tf.data.Dataset containing the test
        set for the subject.

    Returns
    -------
    dict
        Mapping between a subject and the loss/evaluation score on the test set
    """
    evaluation = {}
    for subject, ds_test in test_dict.items():
        logging.info(f"Scores for subject {subject}:")
        results = model.evaluate(ds_test, verbose=2)
        metrics = model.metrics_names
        evaluation[subject] = dict(zip(metrics, results))
    return evaluation


if __name__ == "__main__":
    # Parameters
    # Length of the decision window
    window_length = 5 * 64  # 5s
    # Hop length between two consecutive decision windows
    hop_length = int(64*0.5) #0.5 seconds
    epochs = 100
    patience = 5
    batch_size = 1
    only_evaluate = False
    training_log_filename = "training_log.csv"
    results_filename = 'eval.json'


    # Get the path to the config file
    experiments_folder = "C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression"
    task_folder = os.path.dirname(experiments_folder)
    config_path = os.path.join(task_folder, 'util', 'config.json')

    # Load the config
    with open("C:/Users/YLY/Documents/eegAudChallenge/auditory-eeg-challenge-2023-code/task2_regression/util/config.json") as fp:
        config = json.load(fp)

    # Provide the path of the dataset
    # which is split already to train, val, test

    data_folder = os.path.join(config["dataset_folder"], config["split_folder"])
    stimulus_features = ["envelope"]
    features = ["eeg"] + stimulus_features

    # Create a directory to store (intermediate) results
    results_folder = os.path.join(experiments_folder, "results_linear_baseline")
    os.makedirs(results_folder, exist_ok=True)

    # create a simple linear model
    #model = simple_linear_model()
    #model.summary()
    #model_path = os.path.join(results_folder, "model.h5")

    if only_evaluate:
    #    model = tf.keras.models.load_model(model_path)
        print("evaluate")
    else:

        #train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
        # Create list of numpy array files
        #train_generator = RegressionDataGenerator(train_files, window_length)
       # dataset_train = create_tf_dataset(train_generator, window_length, None, hop_length, batch_size)

        # Create the generator for the validation set
        #val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
        #val_generator = RegressionDataGenerator(val_files, window_length)
        #dataset_val = create_tf_dataset(val_generator, window_length, None, hop_length, batch_size)
        # Convert the TensorFlow dataset to a PyTorch dataset
        #train_dataset= CustomDataset(dataset_train,340585)
        #del dataset_train
        #val_dataset = CustomDataset(dataset_val,38357)
       # del dataset_val 
    #### train the model
    # Parse configs. Globals nicer in this case
        with open('train-shallow.json') as f:
            data = f.read()
        config = json.loads(data)
        train_config = config["train_config"]  # training parameters
        global dist_config
        dist_config = config["dist_config"]  # to initialize distributed training
        global wavenet_config
        wavenet_config = config["wavenet_config"]  # to define wavenet
        global diffusion_config
        diffusion_config = config["diffusion_config"]  # basic hyperparameters
        global trainset_config
        trainset_config = config["trainset_config"]  # to load trainset
        global diffusion_hyperparams
        diffusion_hyperparams = calc_diffusion_hyperparams(
            **diffusion_config)  # dictionary of all diffusion hyperparameters

        torch.backends.cudnn.enabled = True
        torch.backends.cudnn.benchmark = True
        train(window_length,hop_length, 1, 0, "test", **train_config)
    
    
    # Evaluate the model on test set
    # Create a dataset generator for each test subject
    #test_files = [x for x in glob.glob(os.path.join(data_folder, "test_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
    # Get all different subjects from the test set
    #subjects = list(set([os.path.basename(x).split("_-_")[1] for x in test_files]))
    #datasets_test = {}
    # Create a generator for each subject
    #for sub in subjects:
    #    files_test_sub = [f for f in test_files if sub in os.path.basename(f)]
    #    test_generator = RegressionDataGenerator(files_test_sub, window_length)
    #    datasets_test[sub] = create_tf_dataset(test_generator, window_length, None, hop_length, 1)

    # Evaluate the model
    #evaluation = evaluate_model(model, datasets_test)

    # We can save our results in a json encoded file
    #results_path = os.path.join(results_folder, results_filename)
    #with open(results_path, "w") as fp:
    #    json.dump(evaluation, fp)
    #logging.info(f"Results saved at {results_path}")


output directory exp\ch64_T100_betaT0.05\logs/checkpoint
EEGWav_diff Parameters: 2.191232M
No valid checkpoint model found, start training from initialization.
total number of training examples: 4375
iteration: 0 	reduced loss: 0.9517078995704651 	loss: 0.9517078995704651
iteration: 100 	reduced loss: 0.9947216510772705 	loss: 0.9947216510772705
iteration: 200 	reduced loss: 1.0172399282455444 	loss: 1.0172399282455444
iteration: 300 	reduced loss: 0.96627277135849 	loss: 0.96627277135849
iteration: 400 	reduced loss: 1.016547441482544 	loss: 1.016547441482544
iteration: 500 	reduced loss: 1.0092779397964478 	loss: 1.0092779397964478
model at iteration 500 is saved
iteration: 600 	reduced loss: 1.0026050806045532 	loss: 1.0026050806045532


KeyboardInterrupt: 